In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import json
import requests
from keras.preprocessing import image
from flask import Flask, request, jsonify
from flask_restful import Resource, Api, reqparse
from flask_cors import CORS, cross_origin
import requests

In [2]:
# 윈도우 = keras 앞부분에 tensorflow를 붙여줘야한다.
# 우분투 = 앞에 붙일 필요 없이 keras만 작성해도 된다.
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

In [3]:
classifier = VGG16()

In [4]:
# 마지막 레이어를 제거하여 추가적인 네트워크 구성이 가능하게 만든다.
last_layer = str(classifier.layers[-1])

In [5]:
# 윈도우는 앞에 tensorflow. 붙이기 / 우분투는 tensorflow. 없어도 된다.
import tensorflow.keras

from tensorflow.keras.layers import Dense

In [6]:
# 추가할 신규 레이어를 작성하는 파트
new_layer = tensorflow.keras.Sequential()

for layer in classifier.layers:
    if str(layer) != last_layer:
        new_layer.add(layer)
        
new_layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [7]:
# 네트워크를 동결시키도록 한다.
for layer in new_layer.layers:
    layer.trainable = False

In [8]:
new_layer.add(
    Dense(256, activation = 'relu')
)
new_layer.add(
    Dense(256, activation = 'relu')
)
new_layer.add(
    Dense(256, activation = 'relu')
)
new_layer.add(
    Dense(128, activation = 'relu')
)
new_layer.add(
    Dense(128, activation = 'relu')
)
new_layer.add(
    Dense(128, activation = 'sigmoid')
)
new_layer.add(
    Dense(64, activation = 'relu')
)
new_layer.add(
    Dense(32, activation = 'relu')
)
new_layer.add(
    Dense(16, activation = 'relu')
)
new_layer.add(
    Dense(8, activation = 'sigmoid')
)
new_layer.add(
    Dense(1, activation = 'sigmoid')
)

new_layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [9]:
new_layer.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [10]:
# 윈도우 [tensorflow.] 추가
from tensorflow.keras.applications.resnet50 import ResNet50

In [11]:
resModel = ResNet50()
resModel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [12]:
# 라플라스 변환을 하면 s-domain이라는 평면으로 이동을 하게 되고
# 라플라스 역변환을 하면 time-domaim(시간축)으로 돌아오게 된다.
from tensorflow.keras.applications.vgg16 import decode_predictions

In [22]:
def rank(file):
    test_A_img = image.load_img(file, target_size = (224, 224))
    
    trans_img = image.img_to_array(test_A_img)
    trans_img = np.expand_dims(trans_img, axis = 0)
    trans_img = preprocess_input(trans_img)
    
    pred = classifier.predict(trans_img)
    label = decode_predictions(pred)
    
    print('top 5!')
    for i in range(5):
        print('%16s (%.2f%%)' % (label[0][i][1], label[0][i][2] * 100))

In [ ]:
# 여기에 flask 모델 추가하는곳으로 예상
# vue에서 flask, axios로 python으로 이미지를 받아온다.
from flask import send_file
from flask import Flask
from flask_cors import CORS, cross_origin

# 분석 결과가 나왔으면 나온 결과를 vue로 연결해서 보여준다.
app = Flask(__name__)

CORS(app, resources={r'*': {'origins': 'http://localhost:8080'}})
@app.route('/img', methods = ['POST'])
def analysis_img():
    print('b')
    nickname = request.get_json()
    print('a')
    data = {"success": False}
    if request.method == 'POST':
        print('POST')
        print(nickname)
        print(nickname['nick'])
        name = nickname['nick']
        gra=rank(name)
        print(gra)
        return jsonify(gra)
    
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2020 15:42:32] "OPTIONS /img HTTP/1.1" 200 -


b
a
POST
{'nick': '6.jpg'}
6.jpg
40960/35363 [==================================] - 0s 2us/step


127.0.0.1 - - [17/Nov/2020 15:42:33] "POST /img HTTP/1.1" 200 -


top 5!
      spider_web (97.00%)
   garden_spider (1.41%)
     barn_spider (0.92%)
black_and_gold_garden_spider (0.37%)
     wolf_spider (0.14%)
None
